Step 0: Import Statements

In [1]:
## Basic module imports
import pandas as pd
import os
import numpy as np
import torchvision.transforms as torchtransforms
from ImageCreationLib import get_jpeg_files
from ImageCreationLib import extract_patches_simple
from PIL import Image, ImageDraw
import json
import shutil
import cv2

Step 1 - obsolete: Prepare the dataset for training

In [3]:
RELPATH_LABELSFILE = ".\\data2\\labels.csv"
ABSPATH_LABELSFILE = os.path.abspath(RELPATH_LABELSFILE)

pd.options.mode.copy_on_write

dataset = pd.read_csv(ABSPATH_LABELSFILE)

dataset['image_path']=dataset['image_path'].apply(lambda x: x.replace("/media/tom/Elements/rgb_dataset_multi_patches","./data2"))
dataset['image_path']=dataset['image_path'].apply(lambda x: os.path.abspath(x))
dataset['image_path']=dataset['image_path'].apply(lambda x: x.replace("/", "\\\\"))
dataset['objects']=dataset['objects'].apply(lambda x: x.replace("\'", "\""))
dataset_filtered = dataset[dataset['objects']!='[]']

dataset_filtered.to_csv('.\\data2\\labels_fullpath.csv', index=False)
    

Step 1-bis: Prepare the dataset for training

In [5]:
RELPATH_LABELSFILE = ".\\data3\\labels_fullpath_reviewed.csv"
ABSPATH_LABELSFILE = os.path.abspath(RELPATH_LABELSFILE)

pd.options.mode.copy_on_write

dataset = pd.read_csv(ABSPATH_LABELSFILE)

dataset['image_path']=dataset['image_path'].apply(lambda x: (".\\data3\\" + os.path.basename(x)))
dataset['image_path']=dataset['image_path'].apply(lambda x: os.path.abspath(x))
dataset['image_path']=dataset['image_path'].apply(lambda x: x.replace("\\", "\\\\"))

dataset.to_csv('.\\data3\\labels_simplepath.csv', index=False)

Method 2: Convert label-studio .JSON to .CSV

In [ ]:
input_json_file = '.\\data2\\project-5-at-2024-05-03-02-49-9b60eb60.json'

with open(input_json_file, 'r') as json_file:
    data = json.load(json_file)

LOWERBOUND_ID = 24353 
UPPERBOUND_ID = 25078 

df = pd.DataFrame(columns=['index', 'pylon', 'image', 'image_path', 'objects', 'original', 'original_coordinates'])

count=0 

for block in data: 
    if LOWERBOUND_ID <= block['id'] <= UPPERBOUND_ID: 
        #Index
        index = count
        
        #Imagepath  
        imagepath_tmp = block['data']['image']
        imagepath_rel = imagepath_tmp.replace("/data/local-files/?d=rgb_dataset_multi_patches","./data2")
        imagepath_abs = os.path.abspath(imagepath_rel)
        imagepath_final = imagepath_abs.replace("\\", "\\\\")        
        
        #Image
        image = os.path.basename(imagepath_tmp)
        
        #Create the boxes
        annotations = block['annotations']
        results=annotations[0]['result']
        
        list_of_boxes=[]
        
        ORIGINAL_WIDTH = 512
        ORIGINAL_HEIGHT = 512
       
        for result in results:
            x = result['value']['x'] / 100.0 * ORIGINAL_WIDTH 
            y = result['value']['y'] / 100.0 * ORIGINAL_HEIGHT 
            pixel_width = result['value']['width'] / 100.0 * ORIGINAL_WIDTH 
            pixel_height = result['value']['height'] / 100.0 * ORIGINAL_HEIGHT
            if (x+pixel_width) <= ORIGINAL_WIDTH and (y+pixel_height) <= ORIGINAL_HEIGHT :
                box = {}
                box['box']=[x, y, x+pixel_width, y+pixel_height]
                box['class']=LabelMapping.labelmapping[result['value']['rectanglelabels'][0]]
                list_of_boxes.append(box)
        
        list_of_boxes_json = json.dumps(list_of_boxes)
        new_data = [index, '', image, imagepath_final, list_of_boxes_json, '', '']
        df.loc[len(df)] = new_data
        count = count+1

df.to_csv('.\\data2\\labels_fullpath_reviewed.csv', index=False) 

Method 3: Copy selection of Images from Location A to Location B

In [ ]:
TARGET_PATH='C:\\\\FilteredImages\\\\'

for index, row in df.iterrows():
    source_file=row['image_path']
    image_name=row['image']
    destination_file=TARGET_PATH+image_name
    print(source_file)
    print(destination_file)
    shutil.copy(source_file, destination_file)

Method 4: Exclude Images from a dataset

In [9]:
list_to_exclude = ["P100_0_D_8452.jpeg",
"P114_1_A_4230.jpeg",
"P114_0_B_3407.jpeg",
"P114_0_A_9988.jpeg",
"P114_0_A_1885.jpeg",
"P114_0_C_175.jpeg",
"P114_0_DI_2656.jpeg",
"P114_2_C_6545.jpeg",
"P114_3_BM_1515.jpeg",
"P114_0_A_8434.jpeg",
"P114_1_BM_8057.jpeg",
"P118_2_C_8501.jpeg",
"P118_0_D_3066.jpeg",
"P118_1_C_2924.jpeg",
"P118_3_C_4358.jpeg",
"P118_0_BI_6387.jpeg",
"P118_1_C_5522.jpeg",
"P118_0_A_7624.jpeg",
"P118_1_C_4228.jpeg",
"P118_0_BI_1280.jpeg",
"P118_9_H3_3567.jpeg",
"P118_10_BS_3909.jpeg",
"P118_0_A_1576.jpeg",
"P118_0_DM_7043.jpeg",
"P118_1_C_3991.jpeg",
"P118_0_H4_4994.jpeg",
"P118_0_D_1619.jpeg",
"P118_3_C_4542.jpeg",
"P118_5_A_1280.jpeg",
"P118_7_H3-DOWN_8872.jpeg",
"P118_0_C_9194.jpeg",
"P118_3_BS_6938.jpeg",
"P118_5_B_2328.jpeg",
"P118_1_C_4318.jpeg",
"P118_2_D_2032.jpeg",
"P118_2_BI_2786.jpeg",
"P118_5_A_5596.jpeg",
"P118_1_BS_8341.jpeg",
"P122_0_H1-UP_6970.jpeg",
"P122_0_A_3345.jpeg",
"P122_0_A_4404.jpeg",
"P122_1_B_6481.jpeg",
"P122_0_A_7160.jpeg",
"P122_0_B_1468.jpeg",
"P122_0_A_4188.jpeg",
"P122_1_B_347.jpeg",
"P128_0_A_296.jpeg",
"P128_0_D_3618.jpeg",
"P128_0_H2_7894.jpeg",
"P128_1_D_1905.jpeg",
"P128_0_C_4498.jpeg",
"P128_3_A_7193.jpeg",
"P128_4_DM_500.jpeg",
"P128_5_A_9427.jpeg",
"P128_0_A_8040.jpeg",
"P128_2_A_5157.jpeg",
"P128_0_C_2863.jpeg",
"P128_1_B_7290.jpeg",
"P128_1_DM_9975.jpeg",
"P128_2_A_1127.jpeg",
"P128_0_A_2885.jpeg",
"P128_3_B_5552.jpeg",
"P128_1_DM_7061.jpeg",
"P128_2_A_8230.jpeg",
"P128_1_B_6974.jpeg",
"P128_0_C_9768.jpeg",
"P128_0_C_9953.jpeg",
"P128_0_B_3031.jpeg",
"P128_1_C_4757.jpeg",
"P128_0_D_5572.jpeg",
"P128_2_D_8480.jpeg",
"P128_3_C_671.jpeg",
"P128_0_C_1891.jpeg",
"P128_3_C_9784.jpeg",
"P128_0_B_9851.jpeg",
"P128_0_A_6841.jpeg",
"P162_1_DM_5546.jpeg",
"P162_3_H1-DOWN_440.jpeg",
"P162_0_C_3195.jpeg",
"P162_0_D_6772.jpeg",
"P162_0_A_340.jpeg",
"P162_0_A_3828.jpeg",
"P162_0_DM_8268.jpeg",
"P162_1_C_3687.jpeg",
"P162_3_H2-DOWN_5008.jpeg",
"P162_4_H2-DOWN_2991.jpeg",
"P162_0_DM_3522.jpeg",
"P162_4_DM_7197.jpeg",
"P162_0_A_9607.jpeg",
"P162_0_DM_1215.jpeg",
"P162_2_DM_2161.jpeg",
"P162_3_DM_9689.jpeg",
"P162_0_BI_5962.jpeg",
"P162_0_D_3892.jpeg",
"P162_0_A_7377.jpeg",
"P162_0_A_3050.jpeg",
"P162_0_D_2303.jpeg",
"P166_0_H3_4109.jpeg",
"P58_0_D_9248.jpeg",
"P58_1_A_6788.jpeg",
"P58_0_D_4498.jpeg",
"P58_0_A_5038.jpeg",
"P58_0_H3_8883.jpeg",
"P58_0_C_1673.jpeg",
"P58_1_A_9205.jpeg",
"P58_1_B_7715.jpeg",
"P58_0_B_6676.jpeg",
"P58_0_A_1213.jpeg",
"P58_0_C_1647.jpeg",
"P58_2_C_9002.jpeg",
"P58_0_B_3171.jpeg",
"P58_1_B_5138.jpeg",
"P58_3_B_3814.jpeg",
"P58_0_B_6021.jpeg",
"P58_0_B_5827.jpeg",
"P58_1_B_4968.jpeg",
"P58_0_A_6656.jpeg",
"P58_1_C_7263.jpeg",
"P58_0_C_3143.jpeg",
"P58_0_H3_3776.jpeg",
"P58_1_C_3089.jpeg",
"P63_0_D_5489.jpeg",
"P63_0_BI_9497.jpeg",
"P63_1_A_4880.jpeg",
"P63_0_C_1096.jpeg",
"P63_2_D_4374.jpeg",
"P63_0_H3_461.jpeg",
"P63_0_BI_4580.jpeg",
"P63_0_A_4050.jpeg",
"P63_1_A_3223.jpeg",
"P63_0_B_8223.jpeg",
"P63_0_D_8413.jpeg",
"P63_0_A_317.jpeg",
"P63_0_H1_3269.jpeg",
"P63_0_D_9456.jpeg",
"P64_0_A_2784.jpeg",
"P64_0_A_4584.jpeg",
"P64_3_A_4762.jpeg",
"P64_0_A_4244.jpeg",
"P64_0_A_5933.jpeg",
"P64_0_D_4472.jpeg",
"P64_1_A_8091.jpeg",
"P64_0_A_4578.jpeg",
"P64_2_BS_9855.jpeg",
"P64_3_A_5694.jpeg",
"P64_0_A_7404.jpeg",
"P64_0_B_694.jpeg",
"P64_1_D_4915.jpeg",
"P64_0_A_2508.jpeg",
"P64_0_A_4056.jpeg",
"P64_0_B_7265.jpeg",
"P64_0_A_3638.jpeg",
"P64_1_D_8151.jpeg",
"P64_2_DM_4426.jpeg",
"P64_2_C_891.jpeg",
"P64_0_B_3643.jpeg",
"P68_3_D_3593.jpeg",
"P68_4_A_9139.jpeg",
"P68_0_D_7574.jpeg",
"P68_1_D_787.jpeg",
"P68_2_D_6899.jpeg",
"P68_0_C_5231.jpeg",
"P68_1_D_8669.jpeg",
"P68_1_A_9292.jpeg",
"P68_0_B_5990.jpeg",
"P68_0_B_10.jpeg",
"P68_1_D_1231.jpeg",
"P68_0_D_5139.jpeg",
"P68_1_D_8770.jpeg",
"P68_0_B_798.jpeg",
"P68_3_B_7655.jpeg",
"P68_2_B_9507.jpeg",
"P68_2_DM_7065.jpeg",
"P68_0_C_6041.jpeg",
"P68_0_B_2838.jpeg",
"P68_0_C_955.jpeg",
"P68_0_C_9322.jpeg",
"P68_3_D_6073.jpeg",
"P68_7_DM_3912.jpeg",
"P68_2_D_9381.jpeg",
"P68_3_D_4057.jpeg",
"P68_4_D_8139.jpeg",
"P68_4_D_8687.jpeg",
"P68_8_DM_9825.jpeg",
"P68_0_C_4021.jpeg",
"P68_0_B_610.jpeg",
"P68_1_H4_2165.jpeg",
"P76_0_H1_3369.jpeg",
"P80_0_D_889.jpeg",
"P80_4_H1_1145.jpeg",
"P80_2_A_2630.jpeg",
"P80_0_D_8810.jpeg",
"P80_2_A_4242.jpeg",
"P80_2_H3_6607.jpeg",
"P80_3_A_8160.jpeg",
"P80_0_B_3033.jpeg",
"P80_2_D_2788.jpeg",
"P80_5_A_3322.jpeg",
"P80_0_A_2093.jpeg",
"P80_2_D_7957.jpeg",
"P80_3_A_2873.jpeg",
"P80_6_D_873.jpeg",
"P80_7_H3_1473.jpeg",
"P80_0_A_3168.jpeg",
"P80_2_A_8958.jpeg",
"P80_4_D_7604.jpeg",
"P80_6_D_2989.jpeg",
"P80_8_A_2994.jpeg",
"P80_1_C_3784.jpeg",
"P80_1_A_6138.jpeg",
"P80_3_C_6108.jpeg",
"P80_4_D_9101.jpeg"]

RELPATH_LABELSFILE = ".\\data3\\labels_simplepath.csv"
ABSPATH_LABELSFILE = os.path.abspath(RELPATH_LABELSFILE)

pd.options.mode.copy_on_write

dataset = pd.read_csv(ABSPATH_LABELSFILE)

dataset_filtered = dataset[~dataset['image'].isin(list_to_exclude)]

dataset_filtered.to_csv('.\\data3\\labels_simplepath_filtered.csv', index=False)




Method 5: Copy Images to be labeled

In [ ]:
list_to_label = ["P114_0_A_9988.jpeg",
"P114_0_C_175.jpeg",
"P114_2_C_6545.jpeg",
"P114_3_BM_1515.jpeg",
"P114_1_BM_8057.jpeg",
"P118_2_C_8501.jpeg",
"P118_1_C_2924.jpeg",
"P118_3_C_4358.jpeg",
"P118_3_BS_6938.jpeg",
"P118_5_B_2328.jpeg",
"P122_0_H1-UP_6970.jpeg",
"P122_0_A_3345.jpeg",
"P122_0_A_4404.jpeg",
"P122_0_B_1468.jpeg",
"P122_0_A_4188.jpeg",
"P122_1_B_347.jpeg",
"P128_0_D_3618.jpeg",
"P128_4_DM_500.jpeg",
"P128_1_B_7290.jpeg",
"P128_2_A_1127.jpeg",
"P128_1_DM_7061.jpeg",
"P128_2_A_8230.jpeg",
"P128_1_B_6974.jpeg",
"P128_0_C_9768.jpeg",
"P128_0_D_5572.jpeg",
"P128_2_D_8480.jpeg",
"P162_3_H1-DOWN_440.jpeg",
"P162_0_A_9607.jpeg",
"P162_0_D_3892.jpeg",
"P162_0_A_3050.jpeg",
"P162_0_D_2303.jpeg",
"P58_0_D_4498.jpeg",
"P58_0_H3_8883.jpeg",
"P58_1_A_9205.jpeg",
"P58_0_B_6676.jpeg",
"P58_0_A_1213.jpeg",
"P58_0_C_1647.jpeg",
"P58_2_C_9002.jpeg",
"P58_0_B_3171.jpeg",
"P58_0_B_6021.jpeg",
"P58_0_B_5827.jpeg",
"P58_0_A_6656.jpeg",
"P58_1_C_7263.jpeg",
"P58_0_C_3143.jpeg",
"P58_0_H3_3776.jpeg",
"P63_0_BI_4580.jpeg",
"P63_0_A_317.jpeg",
"P63_0_H1_3269.jpeg",
"P63_1_A_2984.jpeg",
"P64_0_A_4584.jpeg",
"P64_0_A_4244.jpeg",
"P64_0_A_5933.jpeg",
"P64_0_D_4472.jpeg",
"P64_1_A_8091.jpeg",
"P64_0_A_4578.jpeg",
"P64_2_BS_9855.jpeg",
"P64_1_D_4915.jpeg",
"P64_0_B_7265.jpeg",
"P64_2_C_891.jpeg",
"P68_3_D_3593.jpeg",
"P68_4_A_9139.jpeg",
"P68_0_D_7574.jpeg",
"P68_1_D_787.jpeg",
"P68_2_D_6899.jpeg",
"P68_0_C_5231.jpeg",
"P68_0_B_5990.jpeg",
"P68_0_B_10.jpeg",
"P68_1_D_1231.jpeg",
"P68_0_B_798.jpeg",
"P68_2_B_9507.jpeg",
"P68_2_DM_7065.jpeg",
"P68_0_C_955.jpeg",
"P68_0_C_9322.jpeg",
"P68_2_D_9381.jpeg",
"P68_4_D_8139.jpeg",
"P68_0_C_4021.jpeg",
"P68_1_H4_2165.jpeg",
"P76_0_H1_3369.jpeg"]

RELPATH_LABELSFILE = ".\\data3\\labels_simplepath.csv"
ABSPATH_LABELSFILE = os.path.abspath(RELPATH_LABELSFILE)

pd.options.mode.copy_on_write
dataset = pd.read_csv(ABSPATH_LABELSFILE)
dataset_filtered = dataset[dataset['image'].isin(list_to_label)]

TARGET_PATH=".\\\\data4_photostolabel\\\\"

for index, row in dataset_filtered.iterrows():
    source_file=row['image_path']
    image_name=row['image']
    destination_file=TARGET_PATH+image_name
    shutil.copy(source_file, destination_file)


Method 6: Convert .JSON to .CSV from home-made labelling

In [35]:
input_json_file = '.\\data5\\patches_101_200\\project-7-at-2024-05-17-22-09-27742ad8.json'

with open(input_json_file, 'r') as json_file:
    data = json.load(json_file)

df = pd.DataFrame(columns=['index', 'pylon', 'image', 'image_path', 'objects', 'original', 'original_coordinates'])

index=0 

for block in data: 
    imagepath = block['image']
    parts = os.path.basename(imagepath).split('-')
    imagename='-'.join(parts[1:])
    imagepath_rel = ".\\data5\\patches_101_200\\"+imagename
    imagepath_abs = os.path.abspath(imagepath_rel)
    imagepath_final = imagepath_abs.replace("\\", "\\\\")
    list_of_boxes=[]
    ORIGINAL_WIDTH = 512
    ORIGINAL_HEIGHT = 512
       
    if 'label' in block.keys(): 
        annotations = block['label']
        for annotation in annotations:
            x = annotation['x'] / 100.0 * ORIGINAL_WIDTH
            y = annotation['y'] / 100.0 * ORIGINAL_HEIGHT
            width = annotation['width'] / 100.0 * ORIGINAL_WIDTH 
            height = annotation['height'] / 100.0 * ORIGINAL_HEIGHT 
            box = {}
            box['box']=[x, y, x+width, y+height]
            if (annotation['rectanglelabels'][0]) == 'rust':
                box['class']="1"
            else:
                box['class']="2"
            list_of_boxes.append(box)
            list_of_boxes_json = json.dumps(list_of_boxes)
        new_data = [index, '', imagename, imagepath_final, list_of_boxes_json, '', '']
        #df.loc[len(df)] = new_data
        #index = index+1
        #shutil.copy(imagepath_rel, ".\\data5\\patches_classifier_unhealthy\\"+imagename)
    else: 
        #Copy image the to new location
        shutil.copy(imagepath_rel, ".\\data5\\patches_classifier_healthy\\"+imagename)
        new_data = [index, '', imagename, imagepath_final, list_of_boxes_json, '', '']
        df.loc[len(df)] = new_data
        index = index+1        

df.to_csv('.\\data5\\patches_classifier_healthy\\patches_classifier_healthy_annotations.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '.\\data5\\patches_101_200\\project-7-at-2024-05-17-22-09-27742ad8.json'

Method 7: Merge 2 .CSV files

In [27]:
file1 = ".\\data5\\patches_fullset\\correct_annotations_davy.csv"
file2 = ".\\data5\\patches_fullset\\patches_1_200.csv"

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

combined_df = pd.merge(df1, df2, how='outer')

combined_df.to_csv('.\\data5\\patches_fullset\\all_patches_annotated.csv')

Method 8: Convert a set of Images to 512x512 patches

In [22]:
path = ".\\\\data5\\\\"
output_path = "c:\\\\Users\\\\dgdwi\\\\OneDrive\\\\PostgraduaatAI\\\\Projectwerk\\\\TowerConditionMonitoringV3\\\\data5\\\\"
patchsize = [512, 512]

jpegfiles = get_jpeg_files(path)

patch_size = (512, 512)

for file in jpegfiles: 
    patches = extract_patches_simple(file, patch_size)
    # Write all patches to the target location 
    for key in patches.keys():
        #target_path = TAR_ABSPATH_PATCHES_HEALTHYTOWERS+"\\"+key
        cv2.imwrite(output_path+key, patches[key])



